In [5]:
!pip install datasets

  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached xxhash-3.5.0-cp310-cp310-win_amd64.whl.metadata (13 kB)
  Using cached multiprocess-0.70.16-py310-none-any.whl.metadata (7.2 kB)
  Using cached aiohttp-3.11.12-cp310-cp310-win_amd64.whl.metadata (8.0 kB)
  Using cached aiohappyeyeballs-2.4.6-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached async_timeout-5.0.1-py3-none-any.whl.metadata (5.1 kB)
  Using cached attrs-25.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached frozenlist-1.5.0-cp310-cp310-win_amd64.whl.metadata (14 kB)
  Using cached multidict-6.1.0-cp310-cp310-win_amd64.whl.metadata (5.1 kB)
  Using cached yarl-1.18.3-cp310-cp310-win_amd64.whl.metadata (71 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.3.0-py3-none-an

In [11]:
# 安裝套件
!pip install torchvision torchaudio transformers datasets evaluate accelerate scikit-learn

  Using cached regex-2024.11.6-cp310-cp310-win_amd64.whl.metadata (41 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.5.2-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   ---------- ----------------------------- 2.6/10.0 MB 16.7 MB/s eta 0:00:01
   ------------------------------------- -- 9.4/10.0 MB 25.5 MB/s eta 0:00:01
   ---------------------------------------- 10.0/10.0 MB 19.4 MB/s eta 0:00:00
Using cached regex-2024.11.6-cp310-cp310-win_amd64.whl (274 kB)
Using cached safetensors-0.5.2-cp38-abi3-win_amd64.whl (303 kB)
Using cached tokenizers-0.21.0-cp39-abi3-win_amd64.whl (2.4 MB)


In [13]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.device(0))

True
0


In [15]:
from datasets import load_dataset, Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
)

import random
from sklearn.metrics import f1_score

In [17]:
from datasets import load_dataset

ds = load_dataset("Johnson8187/Chinese_Multi-Emotion_Dialogue_Dataset")

In [19]:
import pandas as pd

df = pd.read_csv("hf://datasets/Johnson8187/Chinese_Multi-Emotion_Dialogue_Dataset/data.csv")
print(df)

                       text emotion
0                 你要不要去吃午餐？    平淡語氣
1                誒誒誒！我甄選上了！    開心語調
2        我幾天身體好像有點不太舒服，肚子好痛    悲傷語調
3     我的小專題組員都不做事，幹!超後悔跟他一組    憤怒語調
4          他們是不是吵架了？不會打起來吧？    平淡語氣
...                     ...     ...
4154   為什麼每次總是這麼複雜，我到底該怎麼辦？    疑問語調
4155      這不應該是這樣，為什麼我總是弄錯？    疑問語調
4156       我沒有聽錯吧？這真的是發生了嗎？    疑問語調
4157       你到底怎麼想的？這是你的計劃嗎？    疑問語調
4158    這真的是我想要的結果嗎？真是太驚訝了！    驚奇語調

[4159 rows x 2 columns]


In [21]:
print(df['emotion'][0])

平淡語氣


In [23]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import f1_score

'''
設定 hyperparameters
'''
model_name = 'google-bert/bert-base-chinese'
max_seq_length = 512
num_labels = 8  # 修改為8，因為您有8種情緒標籤
output_dir = 'output'

# 讀取訓練資料
df = pd.read_csv("hf://datasets/Johnson8187/Chinese_Multi-Emotion_Dialogue_Dataset/data.csv")
sentences = df['text']
emotion_map = {
    "平淡語氣": 0,
    "關切語調": 1,
    "開心語調": 2,
    "憤怒語調": 3,
    "悲傷語調": 4,
    "疑問語調": 5,
    "驚奇語調": 6,
    "厭惡語調": 7
}

# 將文字標籤轉換為數字
labels = df['emotion'].map(emotion_map)

# 載入 tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 建立完整的 Dataset（包含文本和標籤）
dataset = Dataset.from_dict({
    'text': sentences,
    'labels': labels
})

# 切分訓練集和測試集
dataset = dataset.train_test_split(test_size=0.2, seed=42)

# 定義預處理函數
def preprocess_function(examples):
    # 使用 tokenizer 處理文本
    tokenized = tokenizer(
        examples['text'],
        truncation=True,
        padding='max_length',
        max_length=max_seq_length,
        return_tensors=None  # 重要：不要在這裡指定 return_tensors
    )
    # 確保包含標籤
    tokenized['labels'] = examples['labels']
    return tokenized

# 對數據集進行預處理
tokenized_datasets = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset['train'].column_names  # 移除原始欄位
)

# 讀取模型
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels
)

# 設定訓練參數
training_args = TrainingArguments(
    output_dir='./output',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=2,
    learning_rate=0.00005,
    warmup_steps=100,
    weight_decay=0.01,
    evaluation_strategy="steps",  # 修正參數名稱
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True,
    seed=42,
)

# 計算模型評估指標
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average='weighted')  # 使用weighted因為是多分類問題
    return {
        'f1': f1,
    }

# 設定 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)

# 開始訓練
trainer.train()

# 儲存模型
trainer.save_model(output_dir)

# 儲存 tokenizer
tokenizer.save_pretrained(output_dir)


C:\Users\student\anaconda3\envs\dl\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\student\.cache\huggingface\hub\models--google-bert--bert-base-chinese. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Map: 100%|██████████████████████████████████████████████████████████████████| 832/832 [00:00<00:00, 43

Step,Training Loss,Validation Loss,F1
100,No log,0.478698,0.853399


('output\\tokenizer_config.json',
 'output\\special_tokens_map.json',
 'output\\vocab.txt',
 'output\\added_tokens.json',
 'output\\tokenizer.json')

In [24]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    pipeline,
)
from pprint import pprint

model_dir = './output'
model = AutoModelForSequenceClassification.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(model_dir)
pipe = pipeline(task='text-classification', model=model, tokenizer=tokenizer, device=0)

texts = [
	"我每天都能跟她一起上學，我好開心！",
	"最好的朋友要離開臺灣了，以後可能不容易再見面...",
	"我覺得我快不行了...",
	"剛剛收到研究所錄取的通知書！",
	"今年的冬天好像比較晚來。"
]
result = pipe(texts)
pprint(result)

Device set to use cuda:0


[{'label': 'LABEL_2', 'score': 0.8571394681930542},
 {'label': 'LABEL_4', 'score': 0.9074247479438782},
 {'label': 'LABEL_4', 'score': 0.9293628931045532},
 {'label': 'LABEL_2', 'score': 0.6840886473655701},
 {'label': 'LABEL_0', 'score': 0.9081459045410156}]


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from pprint import pprint

model_dir = './output'
model = AutoModelForSequenceClassification.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(model_dir)
pipe = pipeline(task='text-classification', model=model, tokenizer=tokenizer, device=0)

# 建立標籤到情緒的反向映射
label_to_emotion = {str(v): k for k, v in emotion_map.items()}

def predict_with_emotion(texts):
    results = pipe(texts)
    formatted_results = []

    for text, result in zip(texts, results):
        # 從 'LABEL_X' 提取數字
        label_num = result['label'].split('_')[1]
        emotion = label_to_emotion[label_num]
        formatted_results.append({
            'text': text,
            'emotion': emotion,
            'confidence': round(result['score'], 3)
        })

    return formatted_results

# 測試文本
texts = [
	"我每天都能跟她一起上學，我好開心！",
	"最好的朋友要離開臺灣了，以後可能不容易再見面...",
	"我覺得我快不行了...",
	"剛剛收到研究所錄取的通知書！",
	"今年的冬天好像比較晚來。"
]

results = predict_with_emotion(texts)
for result in results:
    print(f"{result['text']} => {result['emotion']} ({result['confidence']})")
